In [37]:
import requests as req
import pandas as pd
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

KEY = 'pk_723acc97dcdd4f938deaaac9dc5ec542'
TICKERS = ['TSLA', 'AAPL', 'MSFT', 'BRK.B', 'JPM', 'FB', 'XOM', 'AMD', 'NVDA', 'DIS']
'''
max, 5y, 2y, 1y, ytd, 6m, 3m, 1m, 1mm (30 min int), 5d, 5dm(10 min int), date
'''
TIMESCALE = '1y'
DATA_FILE = 'data'

In [2]:
def fetch_data(tickers, timescale, key, dataloc=DATA_FILE):
    for ticker in tickers: 
        url = f'https://cloud.iexapis.com/v1/stock/{ticker}/chart/{timescale}?token={key}'
        #url = f'https://sandbox.iexapis.com/stable/stock/{ticker}/chart/{timescale}?token={key}'
        res = req.get(url)
        with open(f'data/{ticker}{timescale}.json', 'w') as f:
            json.dump(res.json(), f)
            f.close()


# fetch_data(TICKERS, TIMESCALE, KEY)

In [3]:
returns = [] 
for price_data in os.listdir(DATA_FILE):
    with open(f'{DATA_FILE}/{price_data}', 'r') as file:
        data = json.load(file)
        file.close()
    # only use adjusted close data
    prices = [days['close'] for days in data] 
    ticker_returns = [] 
    for i in range(1, len(prices)):
        # calculate return from m-1,m pairs
        ticker_return = ((prices[i] - prices[i-1]) / prices[i]) * 100
        ticker_returns.append(ticker_return)
    
    returns.append(ticker_returns)



In [12]:
# number of stocks
N = len(returns)
# days of returns data
T = len(returns[0])
# standardized stock return matrix, rx - mean / std, N x M dimensions
Std = np.zeros(N)
Y = np.zeros((N, T))
for i in range(N): 
    R_bar = np.mean(returns[i])
    sigma = np.std(returns[i])
    Std[i] = sigma
    Y[i] = [((R_i - R_bar)/sigma) for R_i in returns[i]]

In [131]:
# correlation matrix with dimensions M x M
P = np.matmul(Y, np.transpose(Y)) * (1 / T)

In [161]:
pca = PCA(N)
pca.fit(returns)
eigenvalues = pca.explained_variance_ratio_
eigenvectors = pca.components_

251

-1.3660947373317356e-16

[4.58829879e-01 2.20488683e-01 9.81715034e-02 6.05960187e-02
 5.03241054e-02 3.92735223e-02 3.69645739e-02 2.27374209e-02
 1.26142923e-02 1.72656844e-32]
